In [0]:
# Downloading libraries on the cloud machine
!pip install tensorflow-gpu
!pip install keras
!pip install numpy
!pip install scikit-learn

In [0]:
# Loading the libraries
import numpy as np
import pickle
import os
import time

import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, MaxPool2D, Dropout, Flatten, Dense
from keras.layers import Activation, BatchNormalization, GlobalAveragePooling2D, ReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [0]:
# Connection to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
path_data = '/content/gdrive/My Drive/mobile_net_tipe/data_dark'

In [0]:
train_path = path_data + '/train'
valid_path = path_data + '/valid'
test_path = path_data + '/test'

In [0]:
# Creating the data generators
train_batches = ImageDataGenerator(preprocessing_function =
                                   keras.applications.mobilenet_v2.preprocess_input)
                    .flow_from_directory(train_path, target_size = (224, 224),
                                         batch_size = 64, class_mode = 'sparse')
  
valid_batches = ImageDataGenerator(preprocessing_function =
                                   keras.applications.mobilenet_v2.preprocess_input)
                    .flow_from_directory(valid_path, target_size = (224, 224),
                                         batch_size = 30, class_mode = 'sparse')
  
test_batches = ImageDataGenerator(preprocessing_function =
                                  keras.applications.mobilenet_v2.preprocess_input)
                    .flow_from_directory(test_path, target_size = (224, 224),
                                         batch_size = 30, shuffle = False, class_mode = 'sparse')

Found 16800 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [0]:
# Loading th MobileNet graph
mobile = keras.applications.mobilenet_v2.MobileNetV2()

In [0]:
# Printing the representation of the MobileNet
mobile.summary()

In [0]:
# Creating our Deep Neural Network
x = mobile.layers[36].output

x = Conv2D(128, kernel_size = 7, strides = 2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(32, activation = 'relu')(x)
x = Dropout(0.4)(x)

x = Dense(1)(x)
output = ReLU(max_value = 2, name = 'output')(x)

In [0]:
# Creating the model
my_model = Model(inputs = mobile.input, outputs = output)

In [0]:
# Printing the representation of our model
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
# Compiling the model
opt = Adam(lr = 0.0001)

my_model.compile(loss = 'mae', optimizer = opt, metrics = ['accuracy'])

In [0]:
# Creating saving path
filepath_models = '/content/gdrive/My Drive/mobile_net_tipe/models'

model_name = 'model_final_3dir_relu2'

filepath_model = filepath_models + '/' + model_name + '.hdf5'

# If it is not learning a lot during a moment we reduce the learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.6, patience = 2, min_lr=0.00001)

# Keeping only the weights that generalize the best
checkpoint = ModelCheckpoint(filepath_model, monitor='val_loss', save_best_only = True)

# If we see no improvement we stop the training process
early = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights = True)

# Saving the evolution of the loss the accuracy and others...
tb = TensorBoard(log_dir = '/content/gdrive/My Drive/mobile_net_tipe/logs' + '/' + model_name, write_graph = True, write_images = True)

callbacks_list = [reduce_lr, checkpoint, early, tb]

In [0]:
# Training
history = my_model.fit_generator(train_batches, steps_per_epoch = 256,
                                 validation_data = valid_batches,validation_steps = 10, epochs = 4, verbose = 1, callbacks = callbacks_list)

Epoch 1/4
256/256 [==============================] - 93s 364ms/step - loss: 0.0907 - acc: 0.9619 - val_loss: 0.0162 - val_acc: 1.0000
Epoch 2/4
256/256 [==============================] - 90s 350ms/step - loss: 0.0696 - acc: 0.9835 - val_loss: 0.1281 - val_acc: 1.0000
Epoch 3/4
256/256 [==============================] - 90s 350ms/step - loss: 0.0640 - acc: 0.9900 - val_loss: 0.1432 - val_acc: 1.0000
Epoch 4/4
256/256 [==============================] - 90s 350ms/step - loss: 0.0605 - acc: 0.9904 - val_loss: 0.1052 - val_acc: 1.0000


In [0]:
# Predictions on the test set
predictions = my_model.evaluate_generator(test_batches, steps = 10, verbose = 1)

10/10 [==============================] - 1s 116ms/step


In [0]:
# The result : [loss, accuracy]
print(predictions)

[0.019887891970574854, 1.0]
